In [1]:
import pickle

with open("model_output.pkl", "rb") as f:
    preds = pickle.load(f)

In [10]:
# import os
# os.chdir("pretrained-vision-transformer")

import sys
import torch
import yaml
import lightning as L
import numpy as np
sys.path.append('./pretrained/src/')
from eurosat_module import EuroSAT_RGB_DataModule, SentinelTest
from vision_transformer import VisionTransformerPretrained
from lightning.pytorch.callbacks.early_stopping import EarlyStopping
from lightning.pytorch.loggers import TensorBoardLogger

with open('./pretrained/configs/default.yaml', encoding="utf-8") as cf_file:
    config = yaml.safe_load(cf_file.read())
    
    
def get_id(img_path):
    return img_path.split("/")[-1].split("_")[-1].split(".")[0]


datamodule = EuroSAT_RGB_DataModule(config["data_root"], config['batch_size'], config['valid_size'])
datamodule.prepare_data()
datamodule.setup()

labels2ids = datamodule.train_data.dataset.class_to_idx
ids2labels = {v: k for k, v in ids2labels.items()}


def oh2text(one_hot):
    idx = torch.argmax(one_hot).item()
    return ids2labels[idx]


# classArray = []

# for i, (imgs, paths) in enumerate(test_dataloader):

#     img_ids = [get_id(path) for path in paths]
#     preds = model(imgs.to("cuda"))
#     preds = list(preds.logits.detach())
#     preds = [oh2text(pred.detach().cpu()) for pred in preds]
#     classArray += list(zip(img_ids, preds))

# import csv

# with open(f'{execution_start}.csv', 'w', newline='') as csvfile:
#     writer = csv.writer(csvfile)
#     writer.writerow(['test_id', 'label'])

#     # Write each string to a row with its corresponding index as the test_id
#     for i, label in classArray:
#         writer.writerow([i, label])

./drive/MyDrive/AIML24/remote_sensing/otherDatasets/sentinel_2/tif
Dataset ImageFolder
    Number of datapoints: 27000
    Root location: ./drive/MyDrive/AIML24/remote_sensing/otherDatasets/sentinel_2/tif
    StandardTransform
Transform: Compose(
                 <function l2a_approx at 0x7f9019a72840>
                 <function bandselect at 0x7f9019a72d40>
                 ToImage()
                 Resize(size=[224, 224], interpolation=2, antialias=True)
                 ToDtype(scale=True)
                 Normalize(mean=[0.5, 0.5, 0.5], std=[0.5, 0.5, 0.5], inplace=False)
           )
Total number of samples:  27000
Training samples: 18900
Validation samples: 2700
Test samples: 5400


In [11]:
ids2labels

{0: 'AnnualCrop',
 1: 'Forest',
 2: 'HerbaceousVegetation',
 3: 'Highway',
 4: 'Industrial',
 5: 'Pasture',
 6: 'PermanentCrop',
 7: 'Residential',
 8: 'River',
 9: 'SeaLake'}

In [12]:
import sys

sys.path.append("pretrained/src/")
from eurosat_module import SentinelTest

ds = SentinelTest("./drive/MyDrive/AIML24/testset/testset/testset", 2)
ids = [get_id(path) for path in ds.img_paths]

In [13]:
class_array = []
for pred in preds:
    class_array = [*class_array, *[oh2text(pred.loss['logits'][i]) for i in range(pred.loss['logits'].shape[0])]]

class_array = list(zip(ids, class_array))

In [14]:


from datetime import datetime
execution_start = datetime.now().strftime("%m%d%Y-%H%M%S")
import csv
with open(f'{execution_start}.csv', 'w', newline='') as csvfile:
    writer = csv.writer(csvfile)
    writer.writerow(['test_id', 'label'])

    # Write each string to a row with its corresponding index as the test_id
    for i, label in class_array:
        writer.writerow([i, label])


In [ ]:
["a", "b", "c"][2:1]

[]